In [250]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression

In [251]:
data1 = pd.read_csv('../fangchan/lianjia1.csv', encoding='gbk')
data2 = pd.read_csv('../fangchan/lianjia2.csv', encoding='gbk')
data3 = pd.read_csv('../fangchan/lianjia3.csv', encoding='utf-8')
data4 = pd.read_csv('../fangchan/lianjia4.csv', encoding='utf-8')
data5 = pd.read_csv('../fangchan/lianjia5.csv', encoding='utf-8')
data6 = pd.read_csv('../fangchan/lianjia6.csv', encoding='utf-8')
data7 = pd.read_csv('../fangchan/lianjia7.csv', encoding='utf-8')
data = pd.concat([data1, data2, data3, data4, data5, data6, data7])

In [252]:
data = data[['cjzongjia', 'cjdanjia', 'cjxiaoqu', 'cjlouceng', 'cjshijian', 
             'bankuai', 'mendian', 'xingming', 'congyenianxian']]
data = data.dropna()
data['cjdanjia'] = data.cjdanjia.str.replace('元/平', '').astype(np.float32).map(lambda x:round(x/10000, 2))

In [253]:
data = data[data.cjshijian.str.contains('签约时间：')]

In [254]:
data.cjshijian = pd.to_datetime(data.cjshijian.str.replace('签约时间：', ''))

In [255]:
data.set_index('cjshijian', inplace=True)

In [256]:
data.sort_index(inplace=True)

In [257]:
data3 = data['2015-01-01':'2015-09-30']

In [258]:
data4 = data['2015-10-01':'2015-12-31']

In [259]:
data3 = data3.assign(xiaoqu=data3.cjxiaoqu.map(lambda x:x.split()[0]))

In [260]:
all6 = data3.xiaoqu.value_counts()[data3.xiaoqu.value_counts() > 5].index

In [261]:
data3 = data3[data3.xiaoqu.isin(all6)]

In [262]:
data3_m = data3.groupby('xiaoqu').cjdanjia.mean()

In [263]:
data4 = data4.assign(xiaoqu=data4.cjxiaoqu.map(lambda x:x.split()[0]))

In [264]:
data4_m = data4.groupby('xiaoqu').cjdanjia.mean()

In [265]:
xq_up = (data4_m - data3_m) / data3_m

In [266]:
xq_up = xq_up.dropna()

In [267]:
xq_up = xq_up.to_frame()

In [268]:
xq_up = xq_up.assign(zhangfu = (xq_up.cjdanjia > 0.1).astype('int'))

In [269]:
data_taoshu = data3.groupby('xiaoqu').count()['cjxiaoqu'].to_frame().rename(columns={'cjxiaoqu':'taoshu'})

In [270]:
data = pd.merge(xq_up, data_taoshu, left_index=True, right_index=True, how='left')
del data['cjdanjia']

In [271]:
data_danjia = data3.groupby('xiaoqu').cjdanjia.mean().to_frame()

In [272]:
data = pd.merge(data, data_danjia, left_index=True, right_index=True, how='left')

In [273]:
data_zongjia = data3.groupby('xiaoqu').cjzongjia.mean().to_frame()

In [274]:
data = pd.merge(data, data_zongjia, left_index=True, right_index=True, how='left')



In [275]:
data_jjr_num = data3.groupby('xiaoqu').xingming.unique().map(len).to_frame()

In [276]:
data = pd.merge(data, data_jjr_num, left_index=True, right_index=True, how='left')

In [277]:
data_md_num = data3.groupby('xiaoqu').mendian.unique().map(len).to_frame()

In [278]:
data = pd.merge(data, data_md_num, left_index=True, right_index=True, how='left')

In [279]:
data3 = data3.assign(louceng=data3.cjlouceng.map(lambda x:x.split('/')[1]))
data_lc = pd.crosstab(data3.xiaoqu, data3.louceng)
data = pd.merge(data, data_lc, left_index=True, right_index=True, how='left')


建立预测模型

In [280]:
index = np.random.permutation(len(data))

In [281]:
data_train = data.iloc[index[:int(0.7 * len(data))]]

In [282]:
data_test = data.iloc[index[int(0.7 * len(data))]:]

In [283]:
x_train, y_train = data_train[[x for x in data.columns if x != 'zhangfu']], data_train.zhangfu
x_test, y_test = data_test[[x for x in data.columns if x != 'zhangfu']], data_test.zhangfu

In [284]:
model = LogisticRegression()

In [285]:
model.fit(x_train, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [286]:
model.score(x_test, y_test)


0.6607142857142857